### 5.2 Whole Slide Scoring - Prediction Confidence Segmentation

Prediction confidence heatmaps were segmented to generate the CNN model-based whole-slide scores. 

First, a CNN confidence threshold was applied to the heatmaps, with only prediction confidences higher than the threshold retained, indicating the existences of plaques predicted with high confidence. Morphological opening and closing operations were then performed to smooth the masks, and prediction areas exceeding a size threshold set to eliminate CNN-noise provided the predicted quantities of Aβ pathologies (ie. cored plaque, diffuse plaque, and CAA). 

These two hyperparameters, CNN confidence threshold (core: 0.1; diffuse: 0.95; CAA: 0.9) and size threshold (cored: 100; diffuse: 1; CAA: 200), were optimized exclusively by statistical analysis on the combined training and validation sets (32 WSIs). These were then used on the hold-out test set with 30 unseen WSIs to show the generalizability. 

In [1]:
import csv
import glob, os

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import measure
from scipy import stats

from tqdm import tqdm

In [2]:
CSV_DIR = 'data/outputs/CNNscore/WSI_CERAD_AREA.csv'
HEATMAP_DIR = 'data/outputs/heatmaps'
SAVE_DIR = 'data/outputs/CNNscore/'

In [3]:
if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

In [4]:
file = pd.read_csv(CSV_DIR)
filenames = list(file['WSI_ID'])
img_class = ['cored', 'diffuse', 'caa']

# two hyperparameters
confidence_thresholds = [0.1, 0.95, 0.9]
pixel_thresholds = [100, 1, 200]

FileNotFoundError: [Errno 2] File b'data/outputs/CNNscore/WSI_CERAD_AREA.csv' does not exist: b'data/outputs/CNNscore/WSI_CERAD_AREA.csv'

In [ ]:
def count_blobs(mask,
               threshold=1500):
    labels = measure.label(mask, neighbors=8, background=0)
    new_mask = np.zeros(mask.shape, dtype='uint8')
    sizes = []
    
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(mask.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > threshold:
            sizes.append(numPixels)
            new_mask = cv2.add(new_mask, labelMask)

    return sizes, new_mask

In [5]:
new_file = file
for index in [0,1,2]:
    preds = np.zeros(len(file))
    confidence_threshold = confidence_thresholds[index]
    pixel_threshold = pixel_thresholds[index]

    for i, WSIname in tqdm(enumerate(filenames)):
        try:
            heatmap_path = HEATMAP_DIR+'new_WSI_heatmap_{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        except:
            heatmap_path = HEATMAP_DIR+'{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        mask = h[index] > confidence_threshold
        mask = mask.astype(np.float32)

        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

        # Apply morphological closing, then opening operations 
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

        a, m = count_blobs(closing, threshold=pixel_threshold)

        preds[i] = len(a)

    print(confidence_threshold, pixel_threshold)

    new_file['CNN_{}_count'.format(img_class[index])] = preds

new_file.to_csv(SAVE_DIR+'CNN_vs_CERAD.csv', index=False)

NameError: name 'file' is not defined

In [6]:
new_file

NameError: name 'new_file' is not defined